# Using Toolhouse for Tool Use on Fireworks.AI infrastructure
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/orliesaurus/fireworks-cookbook/main/learn/function-calling/notebooks_toolhouse_tools/search_and_generate_toolhouse.ipynb)

[Toolhouse](https://app.toolhouse.ai/) is a comprehensive infrastructure platform designed for developing, deploying, and managing tool-based capabilities in large language models (LLMs).

Toolhouse enables seamless integration of external functionalities—referred to as tools—into LLM workflows.

Key tool capabilities include:
- Extracting structured and unstructured data from web or social media sources
- Programmatically generating images
- Executing code snippets and returning computed outputs

This cookbook provides a technical guide on **enhancing LLMs deployed on** [Fireworks.AI](https://fireworks.ai/) **with tool-based extensions**, eliminating the need for manual coding or explicit prompt engineering.

In this demonstration, we detail the process of configuring an LLM to generate images based on real-time data inputs.

We will leverage Toolhouse with the `firefunction-v2` model, hosted on Fireworks's infrastructure.

This model is optimized for high-precision tool utilization, ensuring robust and accurate execution of integrated tasks.

# 1. Setup

Let's install the dependencies.
We will use the OpenAI SDK which is compatible with Fireworks's API - as long as we set the base_url.

We're also installing Toolhouse's Python SDK to access pre-built, pre-hosted tools.

In [ ]:
!pip install toolhouse openai

In [2]:
# Import packages
import os
from openai import OpenAI
from toolhouse import Toolhouse

To integrate Fireworks and Toolhouse, you'll need to set up two environment variables: `FIREWORKS_API_KEY` and `TOOLHOUSE_API_KEY`. Follow these steps to obtain your API keys:

* **Fireworks API Key**: Get your key by visiting the [Fireworks API Console](https://fireworks.ai/account/api-keys).
* **Toolhouse API Key**: Sign up for Toolhouse using [this link](https://join.toolhouse.ai) to receive FREE credits. You will receive your API key as part of the onboarding step, and you can always, navigate to the [Toolhouse API Keys page](https://app.toolhouse.ai/settings/api-keys) to create and get an API key.
* **Install the X tool**: In your [Toolhouse dashboard](https://app.toolhouse.ai), click Install next to the "Search X" tool ([direct link](https://app.toolhouse.ai/store/search_x)).
* **Install the Image Generation tool**: In your [Toolhouse dashboard](https://app.toolhouse.ai), click Install next to the "Image Generation" tool ([direct link](https://app.toolhouse.ai/store/image_generation_flux)).



# Set up keys in Colab

Once you have signe-up and got both API keys, set them as environment variables to start using Fireworks with Toolhouse.

To do that click on the Key icon in the left menu on Colab and click **"Add new secret".**

Create 2 keys and make sure their **name** is spelled like below:
```
FIREWORKS_API_KEY
TOOLHOUSE_API_KEY
```

Enter the values for each key and switch on the **allow note book access"**

# 2. Setting up the SDKs

Since we've imported the SDKs already, let's initialize them and authenticate ourselves using the API keys we just set.

If you want to check again, your API keys should be now in the left sidebar of Google Colab after clicking the key icon.

In [22]:
from google.colab import userdata
client = OpenAI(base_url = "https://api.fireworks.ai/inference/v1", api_key=userdata.get("FIREWORKS_API_KEY"))

th = Toolhouse(api_key=userdata.get('TOOLHOUSE_API_KEY'), provider="openai")

We will use the `accounts/fireworks/models/firefunction-v2` model.

This model is based on the Mixtral model and was fine-tuned for tool use and function calling:

We can test that we can call the model hosted on Fireworks by running the cell below. It should output some info about NYC.

In [8]:
MODEL = "accounts/fireworks/models/firefunction-v2"
response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "tell me about why new york is awesome"}],
)
print(response.choices[0].message.content)


Well, the Big Apple, the City That Never Sleeps, New York City, there's just so much to love about this place! The energy is electric, the diversity is unparalleled, and the opportunities are endless. From iconic landmarks like the Statue of Liberty, Central Park, and Times Square, to world-class museums like the Met and MoMA, there's always something new to explore and discover. And let's not forget about the food – from classic New York-style pizza and bagels to cutting-edge culinary innovations, your taste buds will never be bored. Plus, with some of the best universities and research institutions in the world, NYC is a hub for learning and innovation. So, why is New York so awesome? Because it's the city where dreams come true, where anything is possible, and where the world comes to live, work, and play!


Likewise, you can use the `th.get_tools()` function to display all of the Toolhouse tools you have installed:

In [ ]:
print('TOOLS AVAILABLE:')
fireworks_tools = th.get_tools()
for tool in th.get_tools():
  for tool in fireworks_tools:
    tool.pop("required", None)
    print(f"Name: {tool['function']['name']}")
    print(f"Type: {tool['type']}")
    print(f"Description: {tool['function']['description']}")

For this demo, we will be using two tools from Toolhouse's store.
- Search X
- Image generation

The first tool lets you search the X social network and the second tool lets you generate an image from a prompt and returns it to the LLM.

# 3. Configure Tool Calls

First we'll configure the user prompt to the LLM.
This is a basic python list with a dictionary within it that takes `role` and a `content`.

We set `role` to `user` because this is a command that we, the user, want the LLM to understand.

Doing so will allow us to give a specific command to the LLM. The LLM will look at what tools we have available and attempt to leverage them to achieve the task.

In [73]:
# User message to the LLM
messages = [
  {
    "role": "user",
    "content": "Find the last three messages from the account named @FireworksAI_HQ on X/Twitter and summarize them in one sentence. Make it funny!",
  }
]

We'll send this message to our LLM hosted on Fireworks.

Because we have tools provided by Toolhouse we can skip having to code our own Search X parser.
The tools are hosted and maintained by the Toolhouse team and are optimized for LLM usage.

In [ ]:
# Fireworks response with Toolhouse tools
response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  # Passes the tools to the model
  tools=fireworks_tools,
)

print("The LLM autonomously decides to use the following tool(s) to achieve its goal of finding data on X/Twitter")
print("Tool selected: ", response.choices[0].message.tool_calls[0].function.name)

As you can see from the output above, the LLM properly identified that we'd like to invoke the 'search_x' tool

You can see the arguments it generated by running the code below, it will break iut down by ID, type and function arguments:

In [ ]:
tools_called = response.choices[0].message.tool_calls
for tool_called in tools_called:
    print(f"ID: {tool_called.id}")
    print(f"Type: {tool_called.type}")
    print(f"Function: {tool_called.function}")
    print('\n')

# 4. Execute Tool Call

Now that we learned how the LLM can generate arguments to pass to a function we'll have to **run the function with those arguments**.

The tool gets executed via the `run_tools` command, with the parameters that were identified in the previous LLM call.


After that we will get the result, and append it to the context, the `messages` variable.

In [ ]:
tool_run = th.run_tools(response)
messages += tool_run

# Here's what the messages variable contains now
print(messages)

Let's see what our messages list looks like:

In [ ]:
for message in tool_run:
  print(f"Role: {message['role']}")
  if 'tool_calls' in message:
    print(f"Tool Calls: {message['tool_calls']}")
  if 'tool_call_id' in message:
    print(f"Tool Call ID: {message['tool_call_id']}")
    print(f"Content: {message['content']}")
    print('\n')

We're now going to pass the whole message chain to the LLM. The LLM will generate a proper response based on our initial prompt:

```
Find the last three messages from the account named @FireworksAI_HQ on X/Twitter and summarize them in one sentence. Make it funny!
```

## 4.1 The Funny Summary

In [61]:
messages_filter = [{k: v for k, v in msg.items() if k not in ["audio", "refusal"]} for msg in messages]
summary_response = client.chat.completions.create(
  model=MODEL,
  messages=messages_filter
)

print('LLM RESPONSE:', summary_response.choices[0].message.content)

LLM RESPONSE: The last three messages from @FireworksAI_HQ on X/Twitter summarize to this funny sentence: "One day, @FireworksAI_HQ just woke up and decided to rule all complex reasoning benchmarks with its new model, f1. But then it got distracted by all the sweet AI Game Night invites and forgot to conquer the world... yet!" 😂


Let's add this last response - a funny summary of three posts retrieved from X - to the history of messages.

To achieve this we're combining two arrays into a new variable called `new_messages`.

In [62]:
new_messages = messages_filter + [
  {
    "role": "assistant",
    "content": summary_response.choices[0].message.content,
  }
]

Next we're going to give a new command to the LLM - generate us an image from this funny caption using the tool provided by Toolhouse.

This tool uses Flux - an image model hosted on Fireworks, then hosts it to a storage location on the img.toolhouse.ai subdomain.

> This shows you that a tool can be used to perform multiple action at once (image generation and hosting), but it's up to you - the user and creator of tools - how to set up every tool for best agentic use.

We've installed this tool in step 1.

In [65]:

generate_image_request = new_messages + [
  {
    "role": "user",
    "content": "Generate an image from the funny summary"
  }
]

generate_image_request_filtered = [{k: v for k, v in msg.items() if k not in ["audio", "refusal"]} for msg in generate_image_request]

# Let's ask the LLM to generate the image
image_response = client.chat.completions.create(
  model=MODEL,
  messages=generate_image_request,
  tools=fireworks_tools
)
tool_run = th.run_tools(image_response)

Now that the LLM generated an image and stored it, we just want it to let the user know that the image is ready and can be viewed.

We're going to - yet again - ask the LLM to take the output of the function call and generate a response.

In [67]:
image_messages = generate_image_request + tool_run
image_messages_filtered = [{k: v for k, v in msg.items() if k not in ["audio", "refusal"]} for msg in image_messages]

last_response = client.chat.completions.create(
  model=MODEL,
  messages=image_messages_filtered,
  tools=fireworks_tools,
)

print('LLM RESPONSE:', last_response.choices[0].message.content)


[{'role': 'user', 'content': 'Find the last three messages from the account named @FireworksAI_HQ on X/Twitter and summarize them in one sentence. Make it funny!'}, {'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_059bQb1Ev2JA4CwMyK7PoTPY', 'function': {'arguments': '{"query": "@FireworksAI_HQ", "max_results": 3}', 'name': 'search_x'}, 'type': 'function', 'index': 0}]}, {'role': 'tool', 'tool_call_id': 'call_059bQb1Ev2JA4CwMyK7PoTPY', 'name': 'search_x', 'content': '<tweet>After #AWSreInvent sessions, it’s time for AI Game Night on Dec 3rd! 🎮 \n\nJoin @MongoDB, @arizeai, @FireworksAI_HQ &amp; @HasuraHQ for games, drinks, and networking with tech innovators.\n\nDon’t miss it! RSVP today. 👇\nhttps://t.co/upX3tWOA7x https://t.co/rDls8EJAKt\nby @Karissa_Wood_ at 2024-11-19T23:21:01.000Z\n Conversation ID: 1859014042683126052\n\n![mongodb.social/6016sSyqq](https://mongodb.social/6016sSyqq\n\nPhoto: ![Image](https://pbs.twimg.com/media/GcyLa5cW0AEl67a.jpg)<tweet>\n\n<tweet>

And there you have it, if you have managed to run all the cells in the correct order you should see a message with a valid URL. If you click it you will be able to see the generated image! The image is pretty random and truly depends on what "funny summary" we generated in step 4.1